In [83]:
import torch
from transformers import (BlipProcessor,
                          BlipForQuestionAnswering)
from PIL import Image
import requests
import numpy as np
import glob
import os
import io
import json
from tqdm.auto import tqdm

# GPU 사용 설정
device = "cuda" if torch.cuda.is_available() else "cpu"

# 모델 로드
model_name = "ybelkada/blip-vqa-base"
processor = BlipProcessor.from_pretrained(model_name)
model = BlipForQuestionAnswering.from_pretrained(model_name).to(device)

print(f"BLIP 모델 로드 완료. (사용 장치: {device})")

BLIP 모델 로드 완료. (사용 장치: cuda)


In [84]:
# 로컬/서버에서 실행할 땐 아래 코드 주석처리 하세요
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [85]:
import os

# --- 1. 환경별 프로젝트 루트 경로 정의 ---
# (경로 1: Google Drive Mount Colab)
COLAB_DRIVE_ROOT = "/content/drive/Othercomputers/내 컴퓨터/데이콘 출판마을 프로젝트/github"
# (경로 2: "서버" 또는 다른 Colab 환경)
SERVER_ROOT = "/content/group5_project"


# --- 2. 환경 감지 및 PROJECT_ROOT 설정 ---
if os.path.exists(COLAB_DRIVE_ROOT):
    # Google Drive Colab 환경
    PROJECT_ROOT = COLAB_DRIVE_ROOT
    print(f"환경 감지: Google Drive Colab")

elif os.path.exists(SERVER_ROOT):
    # "서버" (group5_project) 환경
    PROJECT_ROOT = SERVER_ROOT
    print(f"환경 감지: Server (group5_project)")

# --- 3. 최종 DATA_DIR 설정 ---
# (PROJECT_ROOT가 무엇으로 결정되었든, 그 하위에 data 폴더를 붙임)
DATA_DIR = os.path.join(PROJECT_ROOT, "data")


print(f"최종 DATA_DIR: {DATA_DIR}")

# --- 4. (예시) 나머지 파일 경로 설정 ---
LANDMARK_QA_FILE = os.path.join(DATA_DIR, "landmark_qa.json")
# print(f"JSON 파일 경로: {LANDMARK_QA_FILE}")

환경 감지: Google Drive Colab
최종 DATA_DIR: /content/drive/Othercomputers/내 컴퓨터/데이콘 출판마을 프로젝트/github/data


In [86]:
with open(LANDMARK_QA_FILE, 'r', encoding='utf-8') as f:
    data = json.load(f)
    print(f"Landmark Q&A data loaded from '{LANDMARK_QA_FILE}'.")
    questions = data

Landmark Q&A data loaded from '/content/drive/Othercomputers/내 컴퓨터/데이콘 출판마을 프로젝트/github/data/landmark_qa.json'.


In [87]:
questions

{'피노키오': ['Is there a prominent human-shaped statue in the picture?',
  'If there is a statue, is it a character from a fairy tale?',
  'Does the statue have a particularly long nose?',
  'Is the statue wearing green-colored clothes?',
  "Are there flower patterns on the statue's clothes?",
  'Is the statue holding something?',
  'Is the object the statue is holding a book?',
  'Is the hat the statue is wearing a pointed cone shape?',
  "Are the statue's arms relatively thin and long?",
  'Is the statue holding up a finger with its right hand, like the number 1?'],
 '네모탑': ['Is this object shaped like a tower?',
  'Is this tower shaped like stacked square boxes?',
  'Is the height much longer than the width?',
  'Is this object made of wood?',
  'Is the color reddish-brown?',
  'Does the color look like rusty metal?',
  'Is the surface smooth?',
  'Does the sculpture only use dark colors?',
  'Does this sculpture have many layers?',
  'Is the number of layers exactly 5?',
  'Do the lay

In [112]:
questions_temp = {
    "피노키오": [
        'Is there a prominent human-shaped statue in the picture?',
  'If there is a statue, is it a character from a fairy tale?',
  'Does the statue have a particularly long nose?',
  'Is the statue wearing green-colored clothes?',
  "Are there flower patterns on the statue's clothes?",
  'Is the statue holding something?',
  'Is the object the statue is holding a book?',
  'Is the hat the statue is wearing a pointed cone shape?',
  "Are the statue's arms relatively thin and long?",
  'Is the statue holding up a finger with its right hand, like the number 1?'
    ],
    "네모탑": [
        "Is the main object a sculpture?",
        "Is the sculpture located outdoors?",
        "Is the sculpture made of metal?",
        "Is the sculpture reddish-brown in color?",
        "Does the sculpture's surface appear rusted?",
        "Is the sculpture taller than it is wide?",
        "Does the sculpture have a vertical, tower-like shape?",
        "Is the sculpture an abstract piece of art?",
        "Is the sculpture composed of stacked sections?",
        "Are the sections geometric or box-like?",
        "Are there more than three stacked sections?",
        "Do the stacked sections decrease in size towards the top?",
        "Is the sculpture hollow?",
        "Does the sculpture have large, square cutouts or openings?",
        "Are these openings on the sides of the sections?",
        "Can you see through the openings to the other side?",
        "Does the top-most section have an opening?",
        "Does the bottom-most section have openings that form 'legs'?",
        "Is the sculpture standing directly on the ground?",
        "Does the sculpture have flat surfaces?",
        "Does the sculpture have sharp edges and corners?",
        "Is the sculpture monochromatic (a single main color)?",
        "Is the base of the sculpture wider than the top?",
        "Are all the openings square or rectangular?",
        "Is the sculpture's design based on simple geometric shapes?"
    ],
    "지혜의숲 조각상": [
        "Is the main object a sculpture?",
        "Is the sculpture located indoors?",
        "Is the sculpture human-shaped (an anthropomorphic figure)?",
        "Is the sculpture in a sitting position?",
        "Is the sculpture covered in many bright colors?",
        "Does the sculpture have patterns like flowers?",
        "Does the sculpture have patterns with swirls?",
        "Is the sculpture wearing clothes (like a suit or jumpsuit)?",
        "Is the sculpture holding an object up to its eyes?",
        "Is that object a pair of binoculars?",
        "Are the binoculars also covered in colorful patterns?",
        "Is the sculpture's head also colorful?",
        "Is there an object placed next to the sitting sculpture?",
        "Is this object a suitcase or a piece of luggage?",
        "Is the suitcase also covered in colorful patterns?",
        "Is the sculpture sitting on a large base or platform?",
        "Is this base made of many small, stacked blocks?",
        "Do the blocks in the base appear semi-transparent?",
        "Does the base have text written on it?",
        "Is the text composed of English letters?",
        "Is the sculpture in a room with books?",
        "Are there tall, wooden bookshelves in the room?",
        "Is the floor made of wood?",
        "Does the sculpture have visible hands holding the binoculars?",
        "Does the sculpture have visible feet?"
    ]
}

In [113]:
LANDMARK_NAME = "네모탑" # 탐색할 랜드마크 폴더 이름
landmark_dir = os.path.join(DATA_DIR, LANDMARK_NAME)

In [114]:
# --- 1. 이미지 파일 찾기 ---
image_extensions = ["*.jpg", "*.jpeg", "*.png", "*.webp", "*.jfif"]
image_files = []
for ext in image_extensions:
    image_files.extend(glob.glob(os.path.join(landmark_dir, ext)))

if not image_files:
    print(f"경고: '{landmark_dir}' 폴더에서 이미지를 찾을 수 없습니다. 경로를 확인해주세요.")
else:
    print(f"'{LANDMARK_NAME}' 폴더에서 총 {len(image_files)}개의 이미지를 찾았습니다.")


# --- 2. VQA 실행 및 결과 수집 ---
all_answers_set = set()
image_results_list = [] #  이미지별 결과를 저장할 리스트

questions_list = questions_temp.get(LANDMARK_NAME, [])
total_questions_count = len(questions_list)

if total_questions_count == 0:
    print(f"경고: '{LANDMARK_NAME}'에 대한 질문 리스트가 비어있습니다.")
else:
    # tqdm을 사용하여 진행률 표시
    for img_path in tqdm(image_files, desc=f"'{LANDMARK_NAME}' 이미지 처리 중"):
        try:
            image = Image.open(img_path).convert("RGB")
            img_name = os.path.basename(img_path)

            print("\n" + "="*30)
            print(f"{img_name} 처리 중")
            print("="*30)

            # 이미지 표시
            display(image.resize((300, 300)))

            yes_count = 0
            no_count = 0
            pixel_values = processor(images=image, return_tensors="pt").pixel_values.to(device)

            for question in questions_list:
                inputs = processor(text=question, return_tensors="pt").to(device)

                out = model.generate(
                    pixel_values=pixel_values,
                    input_ids=inputs.input_ids,
                    attention_mask=inputs.attention_mask,
                    max_new_tokens=10
                )
                answer = processor.decode(out[0], skip_special_tokens=True).strip().lower()

                # 상세 로그 출력
                print(f"  Q: {question}")
                print(f"  A: {answer}")

                all_answers_set.add(answer)

                if answer == 'yes':
                    yes_count += 1
                elif answer == 'no':
                    no_count += 1

            # --- 결과 계산 및 저장 ---
            yes_ratio = (yes_count / total_questions_count) if total_questions_count > 0 else 0.0

            print(f"\n  [결과] YES: {yes_count}, NO: {no_count} (Ratio: {yes_ratio:.2%})")

            image_results_list.append({
                "image": img_name,
                "yes_count": yes_count,
                "no_count": no_count,
                "yes_ratio": yes_ratio,
                "total_questions": total_questions_count
            })

        except Exception as e:
            print(f"{img_path} 처리 중 오류 발생: {e}")

print("\n\n" + "="*50)
print(f"VQA 처리 완료. 'image_results_list' 변수에 {len(image_results_list)}개 결과 저장됨.")
print("="*50)

Output hidden; open in https://colab.research.google.com to view.

In [101]:
# --- 최종 요약 리스트 출력 ---

# 이전 셀에서 'image_results_list'가 생성되었는지 확인
if 'image_results_list' not in locals() or not image_results_list:
    print("❌ 오류: 'image_results_list' 변수를 찾을 수 없습니다.")
    print("이전 셀을 먼저 실행하여 VQA 처리를 완료해주세요.")
else:
    print("\n" + "="*50)
    print(f"    '{LANDMARK_NAME}' VQA 이미지별 최종 요약 리스트")
    print("="*50)

    print(image_results_list)
    for result in image_results_list:
        print(f"이미지: {result['image']}")
        print(f"  - YES : {result['yes_count']} / {result['total_questions']}")
        print(f"  - NO  : {result['no_count']}")
        print(f"  - 비율: {result['yes_ratio']:.2%}")
        print("-" * 25)

    print("전체 YES 비율 평균 :", sum([result['yes_ratio'] for result in image_results_list]) / len(image_results_list))


    '네모탑' VQA 이미지별 최종 요약 리스트
[{'image': '1685886446687.jpg', 'yes_count': 16, 'no_count': 4, 'yes_ratio': 0.8, 'total_questions': 20}, {'image': '20210413＿152726.jpg', 'yes_count': 17, 'no_count': 3, 'yes_ratio': 0.85, 'total_questions': 20}, {'image': '20221002_112656.jpg', 'yes_count': 17, 'no_count': 3, 'yes_ratio': 0.85, 'total_questions': 20}, {'image': '22-05-17_DSC06296.jpg', 'yes_count': 16, 'no_count': 4, 'yes_ratio': 0.8, 'total_questions': 20}, {'image': 'IMG_8411.jpg', 'yes_count': 16, 'no_count': 4, 'yes_ratio': 0.8, 'total_questions': 20}, {'image': 'SE-0bb33e5a-8ad9-4466-9e4e-d725573fad47.jpg', 'yes_count': 17, 'no_count': 3, 'yes_ratio': 0.85, 'total_questions': 20}, {'image': 'SE-c1d53031-dc4a-4a98-833b-5b72ac518717.jpg', 'yes_count': 15, 'no_count': 5, 'yes_ratio': 0.75, 'total_questions': 20}, {'image': 'image_4919719771491473087251.jpg', 'yes_count': 13, 'no_count': 7, 'yes_ratio': 0.65, 'total_questions': 20}, {'image': 'wrtFileImageView.jpg', 'yes_count': 15, 'no

In [102]:
# --- 1. 이미지 파일 찾기 ---
image_extensions = ["*.jpg", "*.jpeg", "*.png", "*.webp", "*.jfif"]
image_files = []
for ext in image_extensions:
    image_files.extend(glob.glob(os.path.join(landmark_dir, ext)))

if not image_files:
    print(f"경고: '{landmark_dir}' 폴더에서 이미지를 찾을 수 없습니다. 경로를 확인해주세요.")
else:
    print(f"'{LANDMARK_NAME}' 폴더에서 총 {len(image_files)}개의 이미지를 찾았습니다.")


# --- 2. VQA 실행 및 결과 수집 ---
all_answers_set = set()
image_results_list = [] #  이미지별 결과를 저장할 리스트

questions_list = questions_temp.get("지혜의숲 조각상", [])
total_questions_count = len(questions_list)

if total_questions_count == 0:
    print(f"경고: '{LANDMARK_NAME}'에 대한 질문 리스트가 비어있습니다.")
else:
    # tqdm을 사용하여 진행률 표시
    for img_path in tqdm(image_files, desc=f"'{LANDMARK_NAME}' 이미지 처리 중"):
        try:
            image = Image.open(img_path).convert("RGB")
            img_name = os.path.basename(img_path)

            print("\n" + "="*30)
            print(f"{img_name} 처리 중")
            print("="*30)

            # 이미지 표시
            display(image.resize((300, 300)))

            yes_count = 0
            no_count = 0
            pixel_values = processor(images=image, return_tensors="pt").pixel_values.to(device)

            for question in questions_list:
                inputs = processor(text=question, return_tensors="pt").to(device)

                out = model.generate(
                    pixel_values=pixel_values,
                    input_ids=inputs.input_ids,
                    attention_mask=inputs.attention_mask,
                    max_new_tokens=10
                )
                answer = processor.decode(out[0], skip_special_tokens=True).strip().lower()

                # 상세 로그 출력
                print(f"  Q: {question}")
                print(f"  A: {answer}")

                all_answers_set.add(answer)

                if answer == 'yes':
                    yes_count += 1
                elif answer == 'no':
                    no_count += 1

            # --- 결과 계산 및 저장 ---
            yes_ratio = (yes_count / total_questions_count) if total_questions_count > 0 else 0.0

            print(f"\n  [결과] YES: {yes_count}, NO: {no_count} (Ratio: {yes_ratio:.2%})")

            image_results_list.append({
                "image": img_name,
                "yes_count": yes_count,
                "no_count": no_count,
                "yes_ratio": yes_ratio,
                "total_questions": total_questions_count
            })

        except Exception as e:
            print(f"{img_path} 처리 중 오류 발생: {e}")

print("\n\n" + "="*50)
print(f"VQA 처리 완료. 'image_results_list' 변수에 {len(image_results_list)}개 결과 저장됨.")
print("="*50)

Output hidden; open in https://colab.research.google.com to view.

In [103]:
# --- 최종 요약 리스트 출력 ---

# 이전 셀에서 'image_results_list'가 생성되었는지 확인
if 'image_results_list' not in locals() or not image_results_list:
    print("❌ 오류: 'image_results_list' 변수를 찾을 수 없습니다.")
    print("이전 셀을 먼저 실행하여 VQA 처리를 완료해주세요.")
else:
    print("\n" + "="*50)
    print(f"    '{LANDMARK_NAME}' VQA 이미지별 최종 요약 리스트")
    print("="*50)

    print(image_results_list)
    for result in image_results_list:
        print(f"이미지: {result['image']}")
        print(f"  - YES : {result['yes_count']} / {result['total_questions']}")
        print(f"  - NO  : {result['no_count']}")
        print(f"  - 비율: {result['yes_ratio']:.2%}")
        print("-" * 25)

    print("전체 YES 비율 평균 :", sum([result['yes_ratio'] for result in image_results_list]) / len(image_results_list))


    '네모탑' VQA 이미지별 최종 요약 리스트
[{'image': '1685886446687.jpg', 'yes_count': 4, 'no_count': 16, 'yes_ratio': 0.2, 'total_questions': 20}, {'image': '20210413＿152726.jpg', 'yes_count': 4, 'no_count': 16, 'yes_ratio': 0.2, 'total_questions': 20}, {'image': '20221002_112656.jpg', 'yes_count': 3, 'no_count': 17, 'yes_ratio': 0.15, 'total_questions': 20}, {'image': '22-05-17_DSC06296.jpg', 'yes_count': 2, 'no_count': 18, 'yes_ratio': 0.1, 'total_questions': 20}, {'image': 'IMG_8411.jpg', 'yes_count': 4, 'no_count': 16, 'yes_ratio': 0.2, 'total_questions': 20}, {'image': 'SE-0bb33e5a-8ad9-4466-9e4e-d725573fad47.jpg', 'yes_count': 4, 'no_count': 16, 'yes_ratio': 0.2, 'total_questions': 20}, {'image': 'SE-c1d53031-dc4a-4a98-833b-5b72ac518717.jpg', 'yes_count': 2, 'no_count': 18, 'yes_ratio': 0.1, 'total_questions': 20}, {'image': 'image_4919719771491473087251.jpg', 'yes_count': 3, 'no_count': 17, 'yes_ratio': 0.15, 'total_questions': 20}, {'image': 'wrtFileImageView.jpg', 'yes_count': 2, 'no_cou